In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt 
import seaborn as sns 
import matplotlib.colors

import time
from matplotlib.colors import LinearSegmentedColormap
pd.set_option('display.max_columns', None)


C:\Users\saurabh.rane\AppData\Roaming\Python\Python310\site-packages\matplotlib\projections\__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
from nba_api.stats.static import teams

from nba_api.stats.endpoints import leaguedashplayerptshot, leaguedashplayerstats, synergyplaytypes

#### Get Possession DF

In [3]:
season = '2023-24'

poss_df = leaguedashplayerstats.LeagueDashPlayerStats(season=season, \
                                                  measure_type_detailed_defense='Advanced').get_data_frames()[0]



poss_df = poss_df[['PLAYER_ID', 'PLAYER_NAME', 'POSS', 'GP', 'MIN']]

poss_df = poss_df.groupby(['PLAYER_ID', 'PLAYER_NAME']).sum().reset_index()

#### Get Touch Time DF

In [4]:
def get_touch_time_shots(touch_range):
    df = leaguedashplayerptshot.LeagueDashPlayerPtShot(touch_time_range_nullable = touch_range).get_data_frames()[0]
    
    
    df = df[['PLAYER_ID', 'PLAYER_LAST_TEAM_ID' , 'FGM', 'FGA', 'FG2M', 'FG2A', 'FG3M', 'FG3A']]
    
    shot_type = 'self_created'
    if touch_range == "Touch < 2 Seconds":
        shot_type = 'not_self_created'
        
    df['shot_type'] = shot_type
    return df

In [5]:
touch_range_list = ["Touch < 2 Seconds", "Touch 2-6 Seconds",  "Touch 6+ Seconds"]

In [6]:
dataframes = []

# Loop through touch_range_list and call the function for each value
for touch_value in touch_range_list:
    print(touch_value)
    df = get_touch_time_shots(touch_value)
    dataframes.append(df)
    time.sleep(3)
    
touch_df = pd.concat(dataframes, ignore_index=True)

touch_df = touch_df.groupby(['PLAYER_ID', 'PLAYER_LAST_TEAM_ID', 'shot_type']).agg(
    {**{col: 'first' for col in touch_df.columns if col not in ['PLAYER_ID', 'PLAYER_NAME', 'PLAYER_LAST_TEAM_ID', 'shot_type']},
     **{col: 'sum' for col in touch_df.columns if col not in ['PLAYER_ID', 'PLAYER_NAME', 'PLAYER_LAST_TEAM_ID', 'shot_type']}
    }
).reset_index()

touch_df['efg'] = (touch_df.FGM + touch_df.FG3M*0.5)/touch_df.FGA

suffix_mapping = {'not_self_created': '_not_self_created', 'self_created': '_self_created'}

touch_df = touch_df.pivot_table(index=['PLAYER_ID', 'PLAYER_LAST_TEAM_ID' ], columns='shot_type', \
                                values=['FGM', 'FGA', 'FG2M', 'FG2A', 'FG3M', 'FG3A', 'efg'], aggfunc='sum')

# Flatten the MultiIndex columns
touch_df.columns = [f'{col[0]}{suffix_mapping[col[1]]}' if col[1] in suffix_mapping else col[0] for col in touch_df.columns]

touch_df = touch_df.reset_index()

Touch < 2 Seconds
Touch 2-6 Seconds
Touch 6+ Seconds


In [7]:
touch_df[touch_df.PLAYER_ID == 1631218]

,PLAYER_ID,PLAYER_LAST_TEAM_ID,FG2A_not_self_created,FG2A_self_created,FG2M_not_self_created,FG2M_self_created,FG3A_not_self_created,FG3A_self_created,FG3M_not_self_created,FG3M_self_created,FGA_not_self_created,FGA_self_created,FGM_not_self_created,FGM_self_created,efg_not_self_created,efg_self_created
480,1631218,1610612744,296.0,28.0,206.0,20.0,1.0,0.0,0.0,0.0,297.0,28.0,206.0,20.0,0.693603,0.714286


#### Get Synergy Data

In [8]:
def get_synergy_df(season, play_type):

    synergy_df = synergyplaytypes.SynergyPlayTypes(season = season, play_type_nullable = play_type, per_mode_simple = 'Totals',\
                                      type_grouping_nullable = 'Offensive', player_or_team_abbreviation = 'P')\
                                .get_data_frames()[0]

    synergy_df = synergy_df[['PLAYER_ID', 'PLAYER_NAME', 'PLAY_TYPE', 'POSS', 'PTS']]

    synergy_df = synergy_df.groupby(['PLAYER_ID']).sum().reset_index()

    synergy_df['PPP'] = synergy_df.PTS/synergy_df.POSS

    for column in synergy_df.columns:
        # Check if the column name is not PLAYER_ID
        if column != 'PLAYER_ID':
            # Add the suffix based on the play_type variable
            new_column_name = column + "_" + play_type
            # Rename the column
            synergy_df.rename(columns={column: new_column_name}, inplace=True)

    return synergy_df

In [9]:
synergy_df = poss_df[['PLAYER_ID']]

play_types = ['Cut', 'Handoff', 'Isolation', 'OffScreen', 'Postup', 'PRBallHandler',\
              'PRRollman', 'Spotup', 'Transition']

for play_type in play_types:
    print(play_type)
    synergy_df = pd.merge(synergy_df, get_synergy_df(season, play_type),  how = 'left')
    time.sleep(3)
    

Cut


C:\Users\saurabh.rane\AppData\Local\Temp\ipykernel_30904\933239181.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  synergy_df = synergy_df.groupby(['PLAYER_ID']).sum().reset_index()


Handoff


C:\Users\saurabh.rane\AppData\Local\Temp\ipykernel_30904\933239181.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  synergy_df = synergy_df.groupby(['PLAYER_ID']).sum().reset_index()


Isolation


...

C:\Users\saurabh.rane\AppData\Local\Temp\ipykernel_30904\933239181.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  synergy_df = synergy_df.groupby(['PLAYER_ID']).sum().reset_index()


Spotup


C:\Users\saurabh.rane\AppData\Local\Temp\ipykernel_30904\933239181.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  synergy_df = synergy_df.groupby(['PLAYER_ID']).sum().reset_index()


Transition


C:\Users\saurabh.rane\AppData\Local\Temp\ipykernel_30904\933239181.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  synergy_df = synergy_df.groupby(['PLAYER_ID']).sum().reset_index()


#### Data Cleanse

In [10]:
off_ball_df = pd.merge(poss_df, touch_df, on = 'PLAYER_ID')

off_ball_df = pd.merge(off_ball_df, synergy_df, on = 'PLAYER_ID')



In [11]:

for column in off_ball_df.columns:
    if (column.startswith("FG") or column.startswith("PTS") or column.startswith("POSS_")):
        off_ball_df[column] = (off_ball_df[column] / off_ball_df['POSS']) * 75
        
off_ball_df = off_ball_df[off_ball_df.POSS > 1000]

off_ball_df['headshot_url'] = "https://cdn.nba.com/headshots/nba/latest/1040x760/" + \
            off_ball_df.PLAYER_ID.astype(str) + \
                    ".png"

In [16]:
off_ball_df[off_ball_df.PLAYER_ID == 1630590]

,PLAYER_ID,PLAYER_NAME,POSS,GP,MIN,PLAYER_LAST_TEAM_ID,FG2A_not_self_created,FG2A_self_created,FG2M_not_self_created,FG2M_self_created,FG3A_not_self_created,FG3A_self_created,FG3M_not_self_created,FG3M_self_created,FGA_not_self_created,FGA_self_created,FGM_not_self_created,FGM_self_created,efg_not_self_created,efg_self_created,POSS_Cut,PTS_Cut,PPP_Cut,POSS_Handoff,PTS_Handoff,PPP_Handoff,POSS_Isolation,PTS_Isolation,PPP_Isolation,POSS_OffScreen,PTS_OffScreen,PPP_OffScreen,POSS_Postup,PTS_Postup,PPP_Postup,POSS_PRBallHandler,PTS_PRBallHandler,PPP_PRBallHandler,POSS_PRRollman,PTS_PRRollman,PPP_PRRollman,POSS_Spotup,PTS_Spotup,PPP_Spotup,POSS_Transition,PTS_Transition,PPP_Transition,headshot_url,primary_color
324,1630590,Scotty Pippen Jr.,1103,21,25.1,1610612763,0.883953,8.023572,0.611967,4.147779,3.671804,1.223935,1.563917,0.475975,4.555757,9.247507,2.175884,4.623753,0.649254,0.525735,NaN,NaN,NaN,1.223935,1.087942,0.888889,0.74796,0.407978,0.545455,NaN,NaN,NaN,NaN,NaN,NaN,5.711695,4.963735,0.869048,NaN,NaN,NaN,5.031732,5.575703,1.108108,4.419764,5.099728,1.153846,https://cdn.nba.com/headshots/nba/latest/1040x...,#5D76A9


In [13]:
team_colors = pd.read_csv('C:/Users/saurabh.rane/OneDrive - Slalom/NBA/data_update_scripts/Data/teamColors.csv')
team_colors.rename(columns={'TEAM_ID': 'PLAYER_LAST_TEAM_ID', 'Primary Color': 'primary_color'}, inplace=True)
team_colors = team_colors[['PLAYER_LAST_TEAM_ID', 'primary_color']]

In [14]:
off_ball_df = pd.merge(off_ball_df, team_colors, on = 'PLAYER_LAST_TEAM_ID')

In [15]:
off_ball_df.to_csv('observable_off_ball.csv')

In [157]:
off_ball_df[off_ball_df.shot_type == 'self_created'].sort_values(by = 'FGA', ascending = False).head(25)

AttributeError: 'DataFrame' object has no attribute 'shot_type'

In [ ]:
off_ball_df[off_ball_df.shot_type == 'not_self_created'].sort_values(by = 'FGA', ascending = False).head(25)

In [ ]:
pd.read_csv('https://raw.githubusercontent.com/JaseZiv/worldfootballR_data/master/raw-data/all_leages_and_cups/all_competitions.csv')